In [1]:
# Needed for SQL Alchemy  
from sqlalchemy import create_engine
engine=create_engine('sqlite:///.CourseData',echo=False)

# All query results are returned as Pandas DataFrames 
import pandas as pd

#Used for importing CSVs
import glob
import os

# Needed for %sql Magic
%load_ext sql

import sqlite3


In [2]:
#Create CourseData and CourseDqtaWarehouse Databases
conn = sqlite3.connect("./CourseData.db")
conn = sqlite3.connect("./CourseDataWarehouse.db")

In [3]:
CourseCatalog2017_2018=pd.read_csv("./SourceData/Catalogs/CourseCatalog2017_2018.csv")
CourseCatalog2018_2019=pd.read_csv("./SourceData/Catalogs/CourseCatalog2018_2019.csv")

In [4]:
#Reading and combining all course meetings CSVs

#Get list of folders/files in Source Data folder
folders=glob.glob('./SourceData/*')

#Delete folders/files from list that don't contain course meeting CSV files
del folders[5]
del folders[12]
del folders[16]

#Read CSVs from folders in list and combine into a single dataframe
all_files=[]
for path in folders:
    all_files += glob.glob(os.path.join(path, "course_meetings.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
df_from_each_file = (pd.read_csv(f) for f in all_files)
course_meetings = pd.concat(df_from_each_file, ignore_index=True, sort=True)

course_meetings.term.unique()


array(['Fall2015', 'Fall2014', 'Winter2015', 'Spring2015', 'Summer2015',
       'Fall2016', 'Fall2018', 'Fall2017', 'Spring2019',
       'SpringBreak2017', 'Spring2017', 'Winter2018', 'Winter2016',
       'Spring2016', 'Spring2018', 'Winter2017', 'Summer2017',
       'Summer2018', 'Summer2016'], dtype=object)

In [5]:
#Reading and combining all courses CSVs

#Get list of folders/files in Source Data folder
folders=glob.glob('./SourceData/*')

#Delete folders/files from list that don't contain courses CSV files
del folders[5]
del folders[12]
del folders[16]

#Read CSVs from folders in list and combine into a single dataframe
all_files=[]
for path in folders:
    all_files += glob.glob(os.path.join(path, "courses.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
df_from_each_file = (pd.read_csv(f) for f in all_files)
courses = pd.concat(df_from_each_file, ignore_index=True, sort=True)

courses.term.unique()



array(['Fall2015', 'Fall2014', 'Winter2015', 'Spring2015', 'Summer2015',
       'Fall2016', 'Fall2018', 'Fall2017', 'Spring2019',
       'SpringBreak2017', 'Spring2017', 'Winter2018', 'Winter2016',
       'Spring2016', 'Spring2018', 'Winter2017', 'Summer2017',
       'Summer2018', 'Summer2016'], dtype=object)

In [15]:
CourseCatalog2017_2018.to_sql('CourseCatalog2017_2018',con=engine,if_exists='replace')
CourseCatalog2018_2019.to_sql('CourseCatalog2018_2019',con=engine,if_exists='replace')

In [33]:
#Check length of course_meetings DataFrame
len(course_meetings.index)

317321

In [34]:
#import course_meetings DataFrame to SQL as separate tables because DataFrame is too large to import at once.
#Will use a union later in SQL to combine them

course_meetings[:35000].to_sql('course_meetings1',con=engine,if_exists='replace')
course_meetings[35000:70000].to_sql('course_meetings2',con=engine,if_exists='replace')
course_meetings[70000:105000].to_sql('course_meetings3',con=engine,if_exists='replace')
course_meetings[105000:140000].to_sql('course_meetings4',con=engine,if_exists='replace')
course_meetings[140000:175000].to_sql('course_meetings5',con=engine,if_exists='replace')
course_meetings[175000:210000].to_sql('course_meetings6',con=engine,if_exists='replace')
course_meetings[210000:245000].to_sql('course_meetings7',con=engine,if_exists='replace')
course_meetings[245000:280000].to_sql('course_meetings8',con=engine,if_exists='replace')
course_meetings[280000:315000].to_sql('course_meetings9',con=engine,if_exists='replace')
course_meetings[315000:].to_sql('course_meetings10',con=engine,if_exists='replace')

In [36]:
#Check length of courses DataFrame
len(courses.index)

15937

In [37]:
courses.to_sql('courses',con=engine,if_exists='replace')

In [38]:
# Initialize 
%sql sqlite:///.CourseData

'Connected: @.CourseData'

In [39]:
%%sql

SELECT name, sql FROM sqlite_master
WHERE type='table'
ORDER BY name;

 * sqlite:///.CourseData
Done.


name,sql
CourseCatalog2017_2018,"CREATE TABLE ""CourseCatalog2017_2018"" ( ""index"" BIGINT, program_code TEXT, program_name TEXT, catalog_id TEXT, course_title TEXT, credits TEXT, prereqs TEXT, coreqs TEXT, fees TEXT, attributes TEXT, description TEXT)"
CourseCatalog2018_2019,"CREATE TABLE ""CourseCatalog2018_2019"" ( ""index"" BIGINT, program_code TEXT, program_name TEXT, catalog_id TEXT, course_title TEXT, credits TEXT, prereqs TEXT, coreqs TEXT, fees TEXT, attributes TEXT, description TEXT)"
course_meetings1,"CREATE TABLE course_meetings1 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
course_meetings10,"CREATE TABLE course_meetings10 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
course_meetings2,"CREATE TABLE course_meetings2 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
course_meetings3,"CREATE TABLE course_meetings3 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
course_meetings4,"CREATE TABLE course_meetings4 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
course_meetings5,"CREATE TABLE course_meetings5 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
course_meetings6,"CREATE TABLE course_meetings6 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
course_meetings7,"CREATE TABLE course_meetings7 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
